In [1]:
import numpy as np
import scipy.sparse as sp
from scipy.sparse import csc_matrix as csc
import pandas as pd
from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    explained_variance_score,
    roc_auc_score,
    log_loss,
)
from sklearn.preprocessing import minmax_scale

from constants import (SEED, EVENT_THRESHOLD, DEFAULT_K, DEFAULT_THRESHOLD, LOG_DIR, 
                       DATA_DIR, TEST_DATA_PATH, DATA_OCT, DATA_NOV, USECOLS, USER, ITEM, RATING, PREDICTION)
from utilities.ms_evaluation import (rmse, auc, logloss, precision_at_k, recall_at_k, ndcg_at_k, map_at_k)

In [2]:
NAME = r'BasicMatrixFactorization' 
Y_HAT_PATH = DATA_DIR+r'/'+NAME+r'-y_hat.npz'
TEST_RESULTS_PATH = LOG_DIR+'\\'+NAME+'\\test-results.csv'

In [3]:
log = pd.Series(dtype='float64')
y_hat = sp.load_npz(Y_HAT_PATH)
y = sp.load_npz(TEST_DATA_PATH)
assert y_hat.shape == y.shape, 'The shape of Y and Y_hat must match, otherwise they are not comparable.'
print(f"Shape of the matrices: {y.shape}")
print("Number of non-zero values:")
print(f"Y: {y.nnz:8,}")
print(f"Ŷ: {y_hat.nnz:8,}")

Shape of the matrices: (177592, 44780)
Number of non-zero values:
Y:  552,255
Ŷ:  552,255


In [4]:
# Usually, the CSC is used when there are more rows than columns. (If there are more columns, use CSR instead.)
y_hat = y_hat.tocsc()
y = y.tocsc()
y_nz = np.array(y[y.nonzero()]).reshape(-1)
y_hat_nz = np.array(y_hat[y_hat.nonzero()]).reshape(-1)

# Test run for the recommender engine

In [5]:
input_df = pd.concat([pd.read_csv(DATA_OCT, engine='c', sep=',',usecols=USECOLS)
                ,pd.read_csv(DATA_NOV, engine='c', sep=',',usecols=USECOLS)])
drop_visitors = set(input_df.user_id.value_counts()[input_df.user_id.value_counts()<EVENT_THRESHOLD].index)
input_df = input_df[~input_df.user_id.isin(drop_visitors)]
input_df.reset_index(inplace=True,drop=True)
new_user_id = pd.Series(pd.read_csv(DATA_DIR+r'new_user_id.csv', index_col=1, squeeze=True), dtype='int32')
new_product_id = pd.Series(pd.read_csv(DATA_DIR+r'new_product_id.csv', index_col=1, squeeze=True), dtype='int32')
input_df = input_df[input_df.event_type=='purchase']
input_df = input_df.drop(columns=['event_type'])
purchases = set()

for row in input_df.itertuples(): 
    uid = new_user_id[row.user_id]
    pid = new_product_id[row.product_id]
    purchases.add((uid,pid))    
print(f"Number of purchase events in test the dataset: {len(purchases)}")

Number of purchase events in test the dataset: 547912


In [6]:
df_true = []
rows,cols = y.nonzero()
for row,col in zip(rows,cols):
    if (row,col) in purchases:
        df_true.append([row,col,1])
    else:
        df_true.append([row,col,0])
df_true = pd.DataFrame(data=df_true,columns=[USER, ITEM, RATING])
df_true.head(3)

,userID,itemID,rating
0,0,0,1
1,0,80,1
2,0,291,1


In [7]:
df_pred = []
y_hat = y_hat.todok()
rows,cols = y_hat.nonzero()
for row,col in zip(rows,cols):
    df_pred.append([row,col,y_hat[row,col]])
df_pred = pd.DataFrame(data=df_pred,columns=[USER, ITEM, PREDICTION])
df_pred.head(3)  

,userID,itemID,prediction
0,0,0,-0.160520
1,437,0,-0.017408
2,5451,0,0.066711


# Standard metrics

### Mean Square Error

In [8]:
MSE_sparse = csc.sum(csc.power(y_hat-y,2))/y.nnz
MSE = mean_squared_error(y_nz,y_hat_nz)
print(f"Mean Square Error: {MSE_sparse} (calculated using sparse matrix operations)")
print(f"Mean Square Error: {MSE} (calculated using sklearn.metrics)")
print('Note: The smaller the better.')
log["MSE"]=MSE

Mean Square Error: 1.310757711564404 (calculated using sparse matrix operations)
Mean Square Error: 1.3107577562332153 (calculated using sklearn.metrics)
Note: The smaller the better.


### Root Mean Square Error

In [9]:
RMSE = np.sqrt(MSE)
RMSE_sparse = np.sqrt(MSE_sparse)
print(f"Root Mean Square Error: {RMSE_sparse} (calculated using sparse matrix operations)")
print(f"Root Mean Square Error: {RMSE} (calculated using sklearn.metrics)")
print(f"Root Mean Square Error: {rmse(df_true,df_pred)} (calculated using MS Evaluation method)")
print('Note: The smaller the better.')
log["RMSE"]=RMSE

Root Mean Square Error: 1.144883274209386 (calculated using sparse matrix operations)
Root Mean Square Error: 1.1448832750320435 (calculated using sklearn.metrics)
Root Mean Square Error: 1.150347892413276 (calculated using MS Evaluation method)
Note: The smaller the better.


### Mean Absolute Error

In [10]:
MAE_sparse = csc.sum(abs(y_hat-y))/y.nnz
MAE = mean_absolute_error(y_nz,y_hat_nz)
print(f"Mean Absolute Error: {MAE_sparse} (calculated using sparse matrix operations)")
print(f"Mean Absolute Error: {MAE} (calculated using sklearn.metrics)")
print('Note: The smaller the better.')
log["MAE"]=MAE

Mean Absolute Error: 0.5893737381282198 (calculated using sparse matrix operations)
Mean Absolute Error: 0.589373767375946 (calculated using sklearn.metrics)
Note: The smaller the better.


### R²

In [11]:
R2 = r2_score(y_nz,y_hat_nz)
print(f"Coefficient of determination (R\u00B2): {R2}")
print("Note: The closer to 1 the better.")
log["R-squared"]=R2

Coefficient of determination (R²): -17.556097329188866
Note: The closer to 1 the better.


### Explained variance

In [12]:
explained_variance = explained_variance_score(y_nz,y_hat_nz)
print(f"Explained variance: {explained_variance}")
print("Note: The closer to 1 the better.")
log["explained_variance"]=explained_variance

Explained variance: -16.636564254760742
Note: The closer to 1 the better.


### Arear Under Curve (AUC) - integral area under the receiver operating characteristic curve


In [13]:
# We must convert the predicted ratings into a [0, 1] scale.
df_pred_bin = df_pred.copy()
df_pred_bin[PREDICTION] = minmax_scale(df_pred_bin[PREDICTION].astype(float))
auc_v = auc(df_true,df_pred_bin)
print(f"Arear Under Curve (AUC): {auc_v}")
print("Note: The closer to 1 the better. 0.5 indicates an uninformative classifier")
log["AUC"]=auc_v

Arear Under Curve (AUC): 0.5019304152889557
Note: The closer to 1 the better. 0.5 indicates an uninformative classifier


### Logistic loss (logloss)

In [14]:
logisticloss = logloss(df_true,df_pred_bin)
print(f"Logistic loss (logloss): {logisticloss}")
print("Note: The closer to 0 the better.")
log["logloss"]=logisticloss

Logistic loss (logloss): 0.8080938278170747
Note: The closer to 0 the better.


### Precision @ K

In [15]:
prk = precision_at_k(df_true,df_pred)
print(f"Precision @ {DEFAULT_K}: {prk}")
print("Note: The closer to 1 the better.")
log[f"precision-at-{DEFAULT_K}"]=prk

Precision @ 10: 0.34191752705560396
Note: The closer to 1 the better.


### Recall @ K

In [16]:
rk = recall_at_k(df_true,df_pred)
print(f"Recall @ {DEFAULT_K}: {rk}")
print("Note: The closer to 1 the better.")
log[f"recall-at-{DEFAULT_K}"]=rk

Recall @ 10: 0.9668412602660089
Note: The closer to 1 the better.


### normalized Discounted Cumulative Gain

In [17]:
nDCG = ndcg_at_k(df_true,df_pred)
print(f"normalized Discounted Cumulative Gain (nDCG@{DEFAULT_K}): {nDCG}")
print("Note: The closer to 1 the better.")
log[f"nDCG-at-{DEFAULT_K}"]=nDCG

normalized Discounted Cumulative Gain (nDCG@10): 1.0
Note: The closer to 1 the better.


### mAP (mean Average Precision) 

In [18]:
mAP = map_at_k(df_true,df_pred)
print(f"mean Average Precision (mAP@{DEFAULT_K}): {mAP}")
print("Note: The closer to 1 the better.")
log[f"nDCG-at-{DEFAULT_K}"]=mAP

mean Average Precision (mAP@10): 0.9668412602660089
Note: The closer to 1 the better.


In [19]:
log.to_csv(TEST_RESULTS_PATH, index = True, header=False)
log

MSE                    1.310758
RMSE                   1.144883
MAE                    0.589374
R-squared            -17.556097
explained_variance   -16.636564
AUC                    0.501930
logloss                0.808094
precision-at-10        0.341918
recall-at-10           0.966841
nDCG-at-10             0.966841
dtype: float64